In [ ]:
from typing import List
from collections import Counter

스도쿠 배열에 문제가 없는지 확인

In [ ]:
class SudokuProblem:
    """4x4 스도쿠 문제"""
    def __init__(self, initial: List[List[int]]):
        """초기 상태 설정"""
        self.initial = initial
    
    def actions(self, state: List[List[int]]):
        sudoku_state = 1

        for i in range(4):
            for j in range(4):
                if state[i][j] != 0: # 채워진 칸인 경우
                    if self.check_row(state[i]) == 0 or self.check_col(
                        state[0][j], state[1][j], state[2][j],state[3][j]) == 0 or self.check_square(state, i, j) == 0:
                        sudoku_state = 0
                        return sudoku_state
                    else:
                        return sudoku_state
        
    
    
    def check_row(self, row: List[List[int]]):
        row_dup = Counter(row)
        del row_dup[0]
        if(row_dup[1] >= 2 or row_dup[2] >= 2 or row_dup[3] >= 2 or row_dup[4] >= 2):
            return 0
            
    
    def check_col(self, col_0: List[List[int]], col_1: List[List[int]], col_2: List[List[int]], col_3: List[List[int]]):
        col = []
        #col.append 요소개 여러개여서 에러발생으로 extend 사용
        col.extend([col_0,col_1,col_2,col_3])
        col_dup = Counter(col)
        del col_dup[0]
        if(col_dup[1] >= 2 or col_dup[2] >= 2 or col_dup[3] >= 2 or col_dup[4] >= 2):
            return 0
        
    
    def check_square(self, square_state : List[List[int]], i, j):
        square = []
        #1사분면
        if(i < 2 and j >= 2):
            square.extend([square_state[0][2], square_state[0][3], square_state[1][2], square_state[1][3]])
            square_dup = Counter(square)
            del square_dup[0]
            if(square_dup[1] >= 2 or square_dup[2] >= 2 or square_dup[3] >= 2 or square_dup[4] >= 2):
                return 0
        #2사분면
        if(i < 2 and j < 2):
            square.extend([square_state[0][0], square_state[0][1], square_state[1][0], square_state[1][1]])
            square_dup = Counter(square)
            del square_dup[0]
            if(square_dup[1] >= 2 or square_dup[2] >= 2 or square_dup[3] >= 2 or square_dup[4] >= 2):
                return 0
        #3사분면
        if(i >= 2 and j < 2):
            square.extend([square_state[2][0], square_state[2][1], square_state[3][0], square_state[3][1]])
            square_dup = Counter(square)
            del square_dup[0]
            if(square_dup[1] >= 2 or square_dup[2] >= 2 or square_dup[3] >= 2 or square_dup[4] >= 2):
                return 0
        #4사분면
        if(i >= 2 and j >= 2):
            square.extend([square_state[2][2], square_state[2][3], square_state[3][2], square_state[3][3]])
            square_dup = Counter(square)
            del square_dup[0]
            if(square_dup[1] >= 2 or square_dup[2] >= 2 or square_dup[3] >= 2 or square_dup[4] >= 2):
                return 0

A* 알고리즘 구현

In [ ]:
class A_Star_Solver:
    def __init__(self, initial: List[List[int]]):
        """초기 상태 설정"""
        self.initial = initial
    
    def solver(self, state: List[List[int]]):
        save_sudoku = state
        #print(save_sudoku)
        zero = []
        for i in range(4):
            for j in range(4):
                zero.append(state[i][j])
        zero_count = Counter(zero)
        
        print("수정해야 하는 횟수", zero_count.most_common()[0][1])
        
        
        for i in range (zero_count[0]):
            print()
            print(i+1, " 번째 스도쿠 수정")
            road_value = []
            for i in range(4):
                for j in range(4):
                    if save_sudoku[i][j] == 0:
                        road_value.append(self.heuristic(state, i, j))
                    else:
                        road_value.append(0)
                        
            #가장 작은 휴리스틱 값 찾기
            min = 100
            for i in range(16):
                if road_value[i] != 0 and road_value[i] < min:
                    min = road_value[i]
                    
            heuri_min = self.min_heuristic(road_value, min)
            #(heuri_min)
            
            row = heuri_min//4
            col = heuri_min%4
            
            
            
            without = self.without_num(state, row, col)
            
            print("추가될 위치", row+1, "행", col+1, "열")
            print("추가될 값 : ", without)
            #print(without)
            state[row][col] = without
            
            for i in range(4):
                for j in range(4):
                    
                    print(state[i][j], end=" ")
                print()
                
        print("탐색 종료")
            
            
            
            
            
    #ROW, COL, SQUARE를 비교하여 어떤 수가 부족한지
    def without_num(self, state: List[List[int]], row, col):
        #number_array = [1, 2, 3, 4]
        
        #print("존재하지 않은 수")
        without_row = self.without_row(state[row])
        #print(without_row)
        without_col = self.without_col(state[0][col], state[1][col], state[2][col],state[3][col])
        #print(without_col)
        without_square = self.without_square(state, row, col)
        #print(without_square)
        
        without_all = without_row + without_col + without_square
        most_count = Counter(without_all)
        
        #print(most_count.most_common()[0][0])
        return most_count.most_common()[0][0]
        
        
    #ROW에서 없는 수 찾기
    def without_row(self, without_state: List[List[int]]):
        number_array = [1, 2, 3, 4]
        remove_zero = []
        many_zero = 0
        for i in range(4):
            remove_zero.append(without_state[i])
         
        many_zero = Counter(remove_zero)
        for i in range(many_zero[0]):
            remove_zero.remove(0)
        
        for i in range(len(remove_zero)):
            number_array.remove(remove_zero[i])
        
        return number_array
    
    #COL에서 없는 수 찾기
    def without_col(self, col_0: List[List[int]], col_1: List[List[int]], col_2: List[List[int]], col_3: List[List[int]]):
        col_array = []
        col_array.extend([col_0,col_1,col_2,col_3])
        
        
        number_array = [1, 2, 3, 4]
        many_zero = 0
        
        many_zero = Counter(col_array)
        for i in range(many_zero[0]):
            col_array.remove(0)
            
        
        for i in range(len(col_array)):
            #if col_array[i] in number_array:
                number_array.remove(col_array[i])
        
            
        return number_array
    
    
    # 2x2 SQUARE에서 없는 수 찾기
    def without_square(self, state: List[List[int]], row, col):
        square = []
        number_array = [1, 2, 3, 4]
        many_zero = 0
        #1사분면
        if(row < 2 and col >= 2):
            square.extend([state[0][2], state[0][3], state[1][2], state[1][3]])
            many_zero = Counter(square)
            
            for i in range(many_zero[0]):
                square.remove(0)
                
            for i in range(len(square)):
                number_array.remove(square[i])
                
            return number_array
        
        #2사분면
        if(row < 2 and col < 2):
            square.extend([state[0][0], state[0][1], state[1][0], state[1][1]])
            many_zero = Counter(square)
            
            for i in range(many_zero[0]):
                square.remove(0)
                
            for i in range(len(square)):
                number_array.remove(square[i])
                
            return number_array
        
        #3사분면
        if(row >= 2 and col < 2):
            square.extend([state[2][0], state[2][1], state[3][0], state[3][1]])
            many_zero = Counter(square)
            
            for i in range(many_zero[0]):
                square.remove(0)
                
            for i in range(len(square)):
                number_array.remove(square[i])
                
            return number_array
            
        #4사분면
        if(row >= 2 and col >= 2):
            square.extend([state[2][2], state[2][3], state[3][2], state[3][3]])
            many_zero = Counter(square)
            
            for i in range(many_zero[0]):
                square.remove(0)
                
            for i in range(len(square)):
                number_array.remove(square[i])
                
            return number_array
            
    
                        
    #휴리스틱 가장 작은 값과 일치하는 배열 순서
    def min_heuristic(self, state: List[List[int]], min):
        for i in range(16):
            if state[i] == min:
                state[i] = min
                return i            
                        
                        
                            
    #휴리스틱 값 찾기    
    def heuristic(self, state: List[List[int]], row, col):
        heuri_row = self.many_row(state[row])
        heuri_col = self.many_col(state[0][col], state[1][col], state[2][col],state[3][col])
        heuri_square = self.many_square(state, row, col)
        return heuri_row+heuri_col+heuri_square
        
        
    #행에 해당하는 휴리스틱 값 계산   
    def many_row(self, row: List[List[int]]):
        row_zero = Counter(row)
        return row_zero[0]
    
    #열에 해당하는 휴리스틱 값 계산
    def many_col(self, col_0: List[List[int]], col_1: List[List[int]], col_2: List[List[int]], col_3: List[List[int]]):
        col_array = []
        col_array.extend([col_0,col_1,col_2,col_3])
        col_zero = Counter(col_array)
        return col_zero[0]
    
    #어느 사분면에 존재하는지 찾고 해당 사분면의 휴리스틱 값 계산
    def many_square(self, square_state : List[List[int]], i, j):
        square = []
        #1사분면
        if(i < 2 and j >= 2):
            square.extend([square_state[0][2], square_state[0][3], square_state[1][2], square_state[1][3]])
            square_dup = Counter(square)
            return square_dup[0]
        #2사분면
        if(i < 2 and j < 2):
            square.extend([square_state[0][0], square_state[0][1], square_state[1][0], square_state[1][1]])
            square_dup = Counter(square)
            return square_dup[0]
        #3사분면
        if(i >= 2 and j < 2):
            square.extend([square_state[2][0], square_state[2][1], square_state[3][0], square_state[3][1]])
            square_dup = Counter(square)
            return square_dup[0]
        #4사분면
        if(i >= 2 and j >= 2):
            square.extend([square_state[2][2], square_state[2][3], square_state[3][2], square_state[3][3]])
            square_dup = Counter(square)
            return square_dup[0]
        

4x4 스도쿠를 입력하는 곳

In [ ]:
initial_state = [
    [1, 0, 4, 0],
    [0, 0, 0, 1],
    [3, 0, 0, 0],
    [0, 2, 0, 3]
]

스도쿠에 문제가 있으면 에러발생

In [ ]:

problem = SudokuProblem(initial_state)

available_actions = problem.actions(initial_state)

if available_actions == 0:
    print("스도쿠 에러")


문제 없으면 해를 찾아줌

In [ ]:
if available_actions == 1:
    print("스도쿠 문제 없음, A* 탐색 시작")
    solve = A_Star_Solver(initial_state)
    
    available_solve = solve.solver(initial_state)